In [18]:
library(tidyverse)
library(ggplot2)
library(forecast)
library(astsa)
library(xts)
library(tseries)
library(fpp2)
library(fma)
library(lubridate)
library(TSstudio)
library(quantmod)
library(tidyquant)
library(plotly)
library(gridExtra)
library(readxl)
library(imputeTS)
library(zoo)
library(knitr)
library(kableExtra)
library(patchwork)

In [19]:
#Import dataset
df_bus_passengers_PC <- read_csv('./data/df_bus_passengers_PC.csv')

# Import dataset
df_oil_price_monthly_PC <- read_csv('./data/df_oil_price_monthly_PC.csv')

# Create Date
df_oil_price_monthly_PC <- df_oil_price_monthly_PC %>%
  mutate(date2 = make_date(year(date), month(date), 01))

# Check maximum starting date between datasets
if(min(df_bus_passengers_PC$DATE) >= min(df_oil_price_monthly_PC$date))
{
    min_date <- min(df_bus_passengers_PC$DATE)
}else 
    {
        min_date <-min(df_oil_price_monthly_PC$date)
    }

# Keep relevant columns
df_oil_price_monthly_PC <- df_oil_price_monthly_PC %>% select('date2', 'adjusted')

# Rename columns
names(df_bus_passengers_PC) <- c('DATE', 'bus_passengers')

# Rename columns
names(df_oil_price_monthly_PC) <- c('DATE', 'oil_price')

# Filter starting date
df_bus_passengers_PC <- df_bus_passengers_PC %>% filter(DATE >= min_date)

# Filter starting date
df_oil_price_monthly_PC <- df_oil_price_monthly_PC %>% filter(DATE >= min_date)

# Combine datasets
dd <- merge(df_bus_passengers_PC, df_oil_price_monthly_PC, by.x = "DATE", by.y = "DATE", all = TRUE)

# Order by Date sort ascending
dd <- dd %>% arrange(DATE)

# Create the time series object
dd.ts <- ts(dd, star=decimal_date(min_date), frequency = 12)


Rows: 216 Columns: 2
-- Column specification --------------------------------------------------------
Delimiter: ","
dbl  (1): Value
date (1): DATE

i Use `spec()` to retrieve the full column specification for this data.
i Specify the column types or set `show_col_types = FALSE` to quiet this message.
Rows: 233 Columns: 2
-- Column specification --------------------------------------------------------
Delimiter: ","
dbl  (1): adjusted
date (1): date

i Use `spec()` to retrieve the full column specification for this data.
i Specify the column types or set `show_col_types = FALSE` to quiet this message.


In [20]:
fit.reg <- lm(bus_passengers ~ oil_price, data=dd)

#summary(fit.reg)

res.fit <- ts(residuals(fit.reg),star=decimal_date(min_date),frequency = 12)


In [21]:

xt <- res.fit
s <- 12

#write a funtion
SARIMA.c = function(p1,p2,q1,q2,P1,P2,Q1,Q2,data){

    temp <- c()
    d <- 0
    D <- 1
    s <- 12
    n <- 40
    
    i <- 1
    temp <- data.frame()
    ls <- matrix(rep(NA,9*n),nrow=n)

    for (p in p1:p2)
    {
        for(q in q1:q2)
        {
            for(P in P1:P2)
            {
                for(Q in Q1:Q2)
                {
                    if(p+d+q+P+D+Q<=12)
                    {
                        
                        model<- Arima(data,order=c(p-1,d,q-1),seasonal = c(P-1,D,Q-1))
                        ls[i,] <- c(p-1,d,q-1,P-1,D,Q-1,model$aic,model$bic, model$aicc)
                        i <- i+1
                        #print(i)
                    }
                }
            }
        }   
    }
    
    temp <- as.data.frame(ls)
    names(temp) <- c("p","d","q","P","D","Q","AIC","BIC","AICc")
    temp <- na.omit(temp)

    temp
    #knitr::kable(temp)
}

temp <- SARIMA.c(p1=1,p2=3,q1=1,q2=3,P1=1,P2=2,Q1=1,Q2=2,data = xt)

temp

,p,d,q,P,D,Q,AIC,BIC,AICc
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,0,0,0,0,1,0,7369.881,7373.199,7369.900
2,0,0,0,0,1,1,7367.156,7373.793,7367.216
3,0,0,0,1,1,0,7368.978,7375.614,7369.037
4,0,0,0,1,1,1,7364.466,7374.420,7364.586
5,0,0,1,0,1,0,7350.861,7357.497,7350.921
6,0,0,1,0,1,1,7345.849,7355.803,7345.969
7,0,0,1,1,1,0,7349.326,7359.281,7349.446
8,0,0,1,1,1,1,7338.160,7351.432,7338.361
9,0,0,2,0,1,0,7337.662,7347.616,7337.782


In [22]:
AIC <- temp[which.min(temp$AIC),]

p1 <- AIC$p
d1 <- AIC$d
q1 <- AIC$q
P1 <- AIC$P
D1 <- AIC$D
Q1 <- AIC$Q

fit <- Arima(xt, order=c(p1, d1, q1), seasonal=c(P1, D1, Q1))
summary(fit)

Series: xt 
ARIMA(1,0,2)(1,1,1)[12] 

Coefficients:
         ar1      ma1     ma2    sar1     sma1
      0.9790  -0.8660  0.2807  0.2446  -0.9400
s.e.  0.0282   0.0696  0.0767  0.0842   0.1083

sigma^2 = 1.334e+14:  log likelihood = -3614.01
AIC=7240.03   AICc=7240.45   BIC=7259.93

Training set error measures:
                    ME     RMSE     MAE         MPE     MAPE      MASE
Training set -892685.9 11088051 8428585 -0.07640613 124.2668 0.6331264
                     ACF1
Training set -0.001287048

In [23]:
xreg <- dd.ts[, "oil_price"]

fit_auto_arima <- auto.arima(dd.ts[, "bus_passengers"], xreg = xreg)

summary(fit_auto_arima)

p2 <- 1
d2 <- 0
q2 <- 0
P2 <- 0
D2 <- 1
Q2 <- 2

Series: dd.ts[, "bus_passengers"] 
Regression with ARIMA(1,0,0)(0,1,2)[12] errors 

Coefficients:
         ar1     sma1     sma2     xreg
      0.5536  -0.3129  -0.3573  7405046
s.e.  0.0970   0.1011   0.0797  2668898

sigma^2 = 1.949e+14:  log likelihood = -3647.03
AIC=7304.06   AICc=7304.36   BIC=7320.65

Training set error measures:
                   ME     RMSE      MAE        MPE     MAPE      MASE
Training set -2113747 13433027 10654542 -0.5820241 2.450645 0.8270612
                   ACF1
Training set -0.2587463

In [36]:
x <- xt

k <- 24
n <- length(x)
rmse1 <- matrix(NA,n-k,12)
rmse2 <- matrix(NA,n-k,12)

st <- tsp(x)[1]+(k-2)/12

for(i in 1:(n-k))
{

  xtrain <- window(x, end = st + i/12)
  xtest <- window(x, start = st + (i+1)/12, end = st + (i+12)/12)

  print(i)
  print(dim(xtrain))
  print(xtest)

  # Model 1: ARIMA(1,0,2)(1,1,1)[12]
  # Model 2: ARIMA(1,0,0)(0,1,2)[12]
  
  fit <- Arima(xtrain, order=c(p1,d1,q1), seasonal=list(order=c(P1,D1,Q1), period=12),
                include.drift=FALSE, method="ML")
  fcast <- forecast(fit, h=12)
  
  fit2 <- Arima(xtrain, order=c(p2,d2,q2), seasonal=list(order=c(P2,D2,Q2), period=12),
                include.drift=FALSE, method="ML")
  fcast2 <- forecast(fit2, h=12)
  
  rmse1[i,1:length(xtest)]  <- sqrt((fcast$mean-xtest)^2)
  rmse2[i,1:length(xtest)] <- sqrt((fcast2$mean-xtest)^2)
  
}



[1] 1
NULL
           Jan       Feb       Mar       Apr       May       Jun       Jul
2004  -3978389  -4182845  50639679  19253248  14407757  -2524308 -13554130
           Aug       Sep       Oct       Nov       Dec
2004   1660107  19260880  37820126   9709239  13617148
[1] 2
NULL
           Jan       Feb       Mar       Apr       May       Jun       Jul
2004            -4182845  50639679  19253248  14407757  -2524308 -13554130
2005  -2393539                                                            
           Aug       Sep       Oct       Nov       Dec
2004   1660107  19260880  37820126   9709239  13617148
2005                                                  
[1] 3
NULL
           Jan       Feb       Mar       Apr       May       Jun       Jul
2004                      50639679  19253248  14407757  -2524308 -13554130
2005  -2393539 -16984926                                                  
           Aug       Sep       Oct       Nov       Dec
2004   1660107  19260880  37820126   

Warning message in window.default(x, ...):
"'end' value not changed"


[1] 182
NULL
           Feb       Mar       Apr       May       Jun       Jul       Aug
2019 -68486802 -33632956 -27231591 -20068112 -61648594 -53750482 -36982253
           Sep       Oct       Nov       Dec
2019 -24970603   6463315 -50794858 -72708753


Warning message in window.default(x, ...):
"'end' value not changed"


[1] 183
NULL
           Mar       Apr       May       Jun       Jul       Aug       Sep
2019 -33632956 -27231591 -20068112 -61648594 -53750482 -36982253 -24970603
           Oct       Nov       Dec
2019   6463315 -50794858 -72708753


...

[1] 190
NULL
           Oct       Nov       Dec
2019   6463315 -50794858 -72708753


Warning message in window.default(x, ...):
"'end' value not changed"


[1] 191
NULL
           Nov       Dec
2019 -50794858 -72708753


Warning message in window.default(x, ...):
"'end' value not changed"


[1] 192
NULL
           Dec
2019 -72708753
